In [358]:
import random
import sys
sys.path.insert(0,'/home/zahradnik/PycharmProjects/master_thesis')

In [359]:
from src.modules import conf, fit, df_columns, evl, preprocess
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np
import haversine as hs
from gpx_converter import Converter
from datetime import datetime
import statsmodels.api as sm
warnings.simplefilter("ignore")

In [360]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [361]:
race_day = datetime.strptime('2022-10-08-10-30', '%Y-%m-%d-%H-%M')

In [362]:
df = Converter(input_file='hradec.gpx').gpx_to_dataframe()
df['enhanced_altitude'] = df.altitude
df.drop("altitude",axis=1,inplace=True)

In [363]:
def calc_dist(pos1: tuple,pos2: tuple) -> int:
    return hs.haversine(pos1, pos2, unit=hs.Unit.METERS)

In [364]:
## DISTANCE
df['dist_diff'] = [0] + [calc_dist(
    (df['latitude'].iloc[x], df['longitude'].iloc[x]),
    (df['latitude'].iloc[x+1], df['longitude'].iloc[x+1])) for x in range(len(df)-1)]

df['distance'] = df['dist_diff'].cumsum()

In [365]:
df["temp"], df["wind_speed"], df["wind_direct"], df["rain"] = preprocess.get_meteo(
    df.latitude.iloc[0],
    df.longitude.iloc[0],
    df.enhanced_altitude.iloc[0],
    race_day
)
df.drop(['latitude','longitude'],axis=1,inplace=True)

In [366]:
df["slope_ascent"], df["slope_descent"] = preprocess.calc_ascent_descent(df)

df["slope_steep"] = preprocess.calc_slope_steep(df)
#
# df = basic_outlier_detect(
#     df=df,
#     activity_type=activity_type
# )

In [367]:
data = fit.load_pcls(
        athlete_name='zahradnik',
        activity_type='running',
        path_to_load=conf["Paths"]["pcl"],
    )

100%|██████████| 386/386 [00:00<00:00, 1507.35it/s]


In [368]:
hradec_df = data[292]

In [369]:
import random
df['heart_rate'] = np.linspace(min(hradec_df.heart_rate),max(hradec_df.heart_rate),len(df))
df['cadence'] =np.linspace(min(hradec_df.cadence),max(hradec_df.cadence),len(df))

In [370]:
train_df = fit.clean_data(pd.concat(data)[df_columns])
speed_train = train_df.enhanced_speed
train_df.drop('enhanced_speed',axis=1,inplace=True)

len(train_df),len(train_df.columns)
train_df['dist_diff'] = preprocess.calc_dist_diff(train_df.distance)

In [371]:
len(train_df.columns),len(df.columns)

(12, 12)

In [372]:
from scipy.ndimage import uniform_filter1d
for x in range(10,110,10):
    train_df[f'moved_heart_rate_{x}'] = uniform_filter1d(train_df.heart_rate, size=x)
    df[f'moved_heart_rate_{x}'] = uniform_filter1d(df.heart_rate, size=x)


In [373]:
from feature_engine.datetime import DatetimeFeatures
train_df['date'] = train_df.index
df['date'] = '2022-10-08-11-30-10'

dtf = DatetimeFeatures(features_to_extract=['month','week','hour','minute','second'])
train_df= dtf.fit_transform(train_df)
df= dtf.fit_transform(df)

In [374]:
from feature_engine.timeseries.forecasting import WindowFeatures
lagged_cols = ['cadence','heart_rate','slope_steep','slope_ascent','slope_descent']
lagged = 18
for lag in range(1,lagged):
    wft = WindowFeatures(variables=lagged_cols,
                         window=lag)
    train_df = wft.fit_transform(train_df)
    df = wft.fit_transform(df)

In [375]:
len(train_df.columns),len(df.columns)

(112, 112)

In [376]:
for colname in train_df.columns:
    if 'mean' in colname:
        train_df[colname].fillna(0,inplace=True)
        df[colname].fillna(0,inplace=True)

In [377]:
from sklearn.linear_model import Ridge, LinearRegression, RidgeCV
clf = RidgeCV()
clf.fit(train_df, speed_train)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]))

In [378]:
y_pred = clf.predict(df)

In [386]:
result = []
for i, j in zip(df.heart_rate,hradec_df.heart_rate):
    result.append(i - j)
result

[-1.0,
 0.1440162271805292,
 -1.7119675456389416,
 -3.5679513184584124,
 -5.423935091277897,
 -7.279918864097368,
 -8.135902636916839,
 -8.99188640973631,
 -9.84787018255578,
 -10.703853955375251,
 -11.559837728194722,
 -13.415821501014193,
 -14.271805273833678,
 -16.12778904665315,
 -19.98377281947262,
 -20.83975659229209,
 -23.69574036511156,
 -25.551724137931032,
 -26.407707910750503,
 -28.263691683569974,
 -28.11967545638946,
 -27.97565922920893,
 -29.8316430020284,
 -33.68762677484787,
 -32.54361054766734,
 -33.39959432048681,
 -36.25557809330628,
 -37.111561866125754,
 -37.96754563894524,
 -38.82352941176471,
 -40.67951318458418,
 -40.53549695740365,
 -41.39148073022312,
 -42.24746450304259,
 -44.103448275862064,
 -44.959432048681535,
 -45.81541582150102,
 -45.67139959432049,
 -46.52738336713996,
 -46.38336713995943,
 -46.2393509127789,
 -46.095334685598374,
 -45.951318458417845,
 -46.807302231237315,
 -46.6632860040568,
 -45.51926977687627,
 -45.37525354969574,
 -45.231237322515